In [1]:
# import the necessary packages
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from imutils.video import VideoStream
from imutils.video import FPS
import time
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

tf.keras.backend.clear_session()

In [2]:

# load our serialized face detector from disk
# fix seed for reproducible results (only works on CPU, not GPU)
seed = 10
np.random.seed(seed=seed)
tf.random.set_seed(seed=seed)

# hyper parameters for model
nb_classes = 2  # number of classes

img_width, img_height = 32, 64  # change based on the shape/structure of your images
batch_size = 4  # try 4, 8, 16, 32, 64, 128, 256 dependent on CPU/GPU memory capacity (powers of 2 values).
nb_epoch = 500  # number of iteration the algorithm gets trained.
learn_rate = 1e-4  # sgd learning rate
momentum = .9  # sgd momentum to avoid local minimum
rotation_ratio = 10  # how aggressive will be the data augmentation/transformation
zoom_range = .05
width_shift_range=0.05
height_shift_range=0.05
shear_range=0.05
fill_mode='nearest'

patience = int(nb_epoch/3)

train_data_dir = './Data_set_2/Data_prueba_0/'  # Inside, each class should have it's own folder
#validation_data_dir = './dataset/'  # each class should have it's own folder
model_path = './model_2/'

In [3]:

#base_model = tf.keras.applications.Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
#base_model = tf.keras.applications.MobileNetV2(input_shape=(img_width, img_height, 1), weights=None, include_top=False)
base_model = tf.keras.applications.NASNetMobile(input_shape=(img_width, img_height, 1), weights=None, include_top=False) 
#base_model = tf.keras.applications.Xception(input_shape=(img_width, img_height, 1), weights=None, include_top=False) 

# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(nb_classes, activation='softmax')(x)

# add your top layer block to your base model
model = tf.keras.Model(base_model.input, predictions)

# # let's visualize layer names and layer indices to see how many layers/blocks to re-train
# # uncomment when choosing based_model_last_block_layer
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all layers of the based model that is already pre-trained.
for layer in base_model.layers:
    layer.trainable = True

print(model.summary())

"""
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(img_width, img_height, 3)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(nb_classes, activation="softmax"),
    ]
)

model.summary()
"""

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 64, 1)]  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 15, 31, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 15, 31, 32)   128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 15, 31, 32)   0           stem_bn1[0][0]                   
______________________________________________________________________________________________

'\nmodel = tf.keras.Sequential(\n    [\n        tf.keras.Input(shape=(img_width, img_height, 3)),\n        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),\n        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),\n        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),\n        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),\n        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),\n        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),\n        tf.keras.layers.Flatten(),\n        tf.keras.layers.Dropout(0.5),\n        tf.keras.layers.Dense(nb_classes, activation="softmax"),\n    ]\n)\n\nmodel.summary()\n'

In [4]:

# Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.
# To save augmentations un-comment save lines and add to your flow parameters.
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=rotation_ratio,
                                   width_shift_range=width_shift_range,
                                   height_shift_range=height_shift_range,
                                   shear_range=shear_range,
                                   zoom_range=zoom_range,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)



train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    color_mode= "grayscale",
                                                    class_mode='binary',
                                                    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(train_data_dir, # same directory as training data
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        shuffle=True,
                                                        color_mode="grayscale",
                                                        class_mode='binary',
                                                        subset='validation') # set as validation data

#opt = tf.keras.optimizers.SGD(learning_rate=learn_rate, momentum = momentum)
opt = tf.keras.optimizers.Adam(learning_rate=learn_rate)
model.compile(optimizer= opt,
              loss='categorical_crossentropy',  # categorical_crossentropy if multi-class classifier
              metrics=['accuracy'])

# save weights of best training epoch: monitor either val_loss or val_acc

top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', patience= patience, verbose=0)
]

# Train Simple CNN
hist = model.fit(train_generator,
                    steps_per_epoch= int(train_generator.samples/batch_size*.8),
                    epochs=nb_epoch,
                    validation_data=validation_generator,
                    validation_steps = int(validation_generator.samples /batch_size*.8),
                    validation_freq=1,
                    callbacks=callbacks_list)



Found 1511 images belonging to 2 classes.
Found 377 images belonging to 2 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 302 steps, validate for 75 steps
Epoch 1/500
301/302 [============================>.] - ETA: 0s - loss: 0.8129 - accuracy: 0.4875
Epoch 00001: val_loss improved from inf to 4.80805, saving model to /home/dlsaavedra/Desktop/Rentadrone.cl-ai-test&SomeCode/model-definition/GPS_Panel/Classifier/model_2/top_model_weights.h5
302/302 [==============================] - 53s 177ms/step - loss: 0.8136 - accuracy: 0.4884 - val_loss: 4.8080 - val_accuracy: 1.0000
Epoch 2/500
301/302 [============================>.] - ETA: 0s - loss: 0.8003 - accuracy: 0.4464
Epoch 00002: val_loss improved from 4.80805 to 1.74620, saving model to /home/dlsaavedra/Desktop/Rentadrone.cl-ai-test&SomeCode/model-definition/GPS_Panel/Classifier/model_2/top_model_weights.h5
302/302 [==============================] - 27s 88ms/step - loss: 0.8011 - accuracy: 0.4466 - val_loss: 1.7462 

Epoch 26/500
301/302 [============================>.] - ETA: 0s - loss: 0.7792 - accuracy: 0.4879
Epoch 00026: val_loss did not improve from 0.74411
302/302 [==============================] - 29s 95ms/step - loss: 0.7789 - accuracy: 0.4880 - val_loss: 0.8188 - val_accuracy: 1.0000
Epoch 27/500
301/302 [============================>.] - ETA: 0s - loss: 0.7606 - accuracy: 0.4485
Epoch 00027: val_loss did not improve from 0.74411
302/302 [==============================] - 29s 96ms/step - loss: 0.7615 - accuracy: 0.4478 - val_loss: 0.7667 - val_accuracy: 1.0000
Epoch 28/500
301/302 [============================>.] - ETA: 0s - loss: 0.7733 - accuracy: 0.4663
Epoch 00028: val_loss did not improve from 0.74411
302/302 [==============================] - 27s 89ms/step - loss: 0.7731 - accuracy: 0.4673 - val_loss: 0.7690 - val_accuracy: 1.0000
Epoch 29/500
301/302 [============================>.] - ETA: 0s - loss: 0.7709 - accuracy: 0.4763
Epoch 00029: val_loss did not improve from 0.74411
302/3

301/302 [============================>.] - ETA: 0s - loss: 0.7825 - accuracy: 0.4331
Epoch 00055: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7834 - accuracy: 0.4333 - val_loss: 1.1013 - val_accuracy: 0.6667
Epoch 56/500
301/302 [============================>.] - ETA: 0s - loss: 0.7827 - accuracy: 0.4228
Epoch 00056: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 82ms/step - loss: 0.7825 - accuracy: 0.4230 - val_loss: 0.9862 - val_accuracy: 0.6067
Epoch 57/500
301/302 [============================>.] - ETA: 0s - loss: 0.7722 - accuracy: 0.4186
Epoch 00057: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7720 - accuracy: 0.4180 - val_loss: 1.1431 - val_accuracy: 0.5800
Epoch 58/500
301/302 [============================>.] - ETA: 0s - loss: 0.7732 - accuracy: 0.4381
Epoch 00058: val_loss did not improve from 0.74411
302/302 [=========

301/302 [============================>.] - ETA: 0s - loss: 0.7928 - accuracy: 0.5083
Epoch 00084: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7913 - accuracy: 0.5075 - val_loss: 0.8730 - val_accuracy: 0.1533
Epoch 85/500
301/302 [============================>.] - ETA: 0s - loss: 0.7559 - accuracy: 0.4971
Epoch 00085: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7546 - accuracy: 0.4971 - val_loss: 1.0342 - val_accuracy: 0.1933
Epoch 86/500
301/302 [============================>.] - ETA: 0s - loss: 0.7884 - accuracy: 0.4672
Epoch 00086: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7869 - accuracy: 0.4673 - val_loss: 1.0285 - val_accuracy: 0.0133
Epoch 87/500
301/302 [============================>.] - ETA: 0s - loss: 0.7754 - accuracy: 0.4950
Epoch 00087: val_loss did not improve from 0.74411
302/302 [=========

301/302 [============================>.] - ETA: 0s - loss: 0.7936 - accuracy: 0.4846
Epoch 00113: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 82ms/step - loss: 0.7947 - accuracy: 0.4855 - val_loss: 0.8096 - val_accuracy: 0.7667
Epoch 114/500
301/302 [============================>.] - ETA: 0s - loss: 0.7775 - accuracy: 0.4938
Epoch 00114: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 82ms/step - loss: 0.7795 - accuracy: 0.4938 - val_loss: 0.8010 - val_accuracy: 0.2600
Epoch 115/500
301/302 [============================>.] - ETA: 0s - loss: 0.7889 - accuracy: 0.5050
Epoch 00115: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 82ms/step - loss: 0.7887 - accuracy: 0.5050 - val_loss: 0.8136 - val_accuracy: 0.2433
Epoch 116/500
301/302 [============================>.] - ETA: 0s - loss: 0.7855 - accuracy: 0.5229
Epoch 00116: val_loss did not improve from 0.74411
302/302 [======

Epoch 142/500
301/302 [============================>.] - ETA: 0s - loss: 0.7810 - accuracy: 0.5960
Epoch 00142: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7796 - accuracy: 0.5965 - val_loss: 0.8038 - val_accuracy: 0.8600
Epoch 143/500
301/302 [============================>.] - ETA: 0s - loss: 0.7846 - accuracy: 0.5553
Epoch 00143: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 82ms/step - loss: 0.7843 - accuracy: 0.5543 - val_loss: 0.7557 - val_accuracy: 0.4967
Epoch 144/500
301/302 [============================>.] - ETA: 0s - loss: 0.7662 - accuracy: 0.5482
Epoch 00144: val_loss did not improve from 0.74411
302/302 [==============================] - 26s 85ms/step - loss: 0.7637 - accuracy: 0.5472 - val_loss: 0.7528 - val_accuracy: 0.4433
Epoch 145/500
301/302 [============================>.] - ETA: 0s - loss: 0.7976 - accuracy: 0.5121
Epoch 00145: val_loss did not improve from 0.74411
3

Epoch 171/500
301/302 [============================>.] - ETA: 0s - loss: 0.7774 - accuracy: 0.5054
Epoch 00171: val_loss did not improve from 0.74411
302/302 [==============================] - 26s 86ms/step - loss: 0.7771 - accuracy: 0.5062 - val_loss: 0.7739 - val_accuracy: 0.9467
Epoch 172/500
301/302 [============================>.] - ETA: 0s - loss: 0.7685 - accuracy: 0.4676
Epoch 00172: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7683 - accuracy: 0.4685 - val_loss: 0.7521 - val_accuracy: 0.9033
Epoch 173/500
301/302 [============================>.] - ETA: 0s - loss: 0.7918 - accuracy: 0.4913
Epoch 00173: val_loss did not improve from 0.74411
302/302 [==============================] - 25s 83ms/step - loss: 0.7938 - accuracy: 0.4930 - val_loss: 0.7620 - val_accuracy: 0.7667
Epoch 174/500
301/302 [============================>.] - ETA: 0s - loss: 0.7850 - accuracy: 0.4755
Epoch 00174: val_loss did not improve from 0.74411
3

In [5]:

model_json = model.to_json()
with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
    json_file.write(model_json)

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

json_file = open(os.path.join(os.path.abspath(model_path), 'model.json'), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)


#Confution Matrix and Classification Report
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = train_generator.class_indices.keys()
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.
Confusion Matrix
[[ 86  82]
 [ 88 121]]
Classification Report
              precision    recall  f1-score   support

    0-Normal       0.49      0.51      0.50       168
     1-Falla       0.60      0.58      0.59       209

    accuracy                           0.55       377
   macro avg       0.55      0.55      0.55       377
weighted avg       0.55      0.55      0.55       377



In [ ]:
num_classes = 10
input_shape = (32, 32, 1)

"""
base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, weights=None, include_top=False)

# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(10, activation='softmax')(x)

# add your top layer block to your base model
model = tf.keras.Model(base_model.input, predictions)
"""

model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [ ]:


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_train = tf.image.resize(x_train, [input_shape[1], input_shape[0]])
x_test = tf.image.resize(x_test, [input_shape[1], input_shape[0]])

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
A = train_generator.next()
Im = A[0][0][:,:,0]
label = A[1][0]
plt.imshow(cv2.resize(Im,(64,128)), cmap = 'gray')
plt.title(label)